In [1]:
import re
import math
import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from functools import partial

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Selected device: {device}')

Selected device: cuda


In [2]:
data = """Родился на улице Герцена, в гастрономе номер двадцать два. Известный экономист, по призванию своему — библиотекарь. В народе — колхозник. В магазине — продавец.
    В экономике, так сказать, необходим. Это, так сказать, система… э-э-э… в составе ста двадцати единиц. Фотографируете Мурманский полуостров и получаете «Те-ле-фун-кен».
    И бухгалтер работает по другой линии — по линии библиотекаря. Потому что не воздух будет, академик будет! Ну вот можно сфотографировать Мурманский полуостров. Можно
    стать воздушным асом. Можно стать воздушной планетой. И будешь уверен, что эту планету примут по учебнику. Значит, на пользу физике пойдёт одна планета. Величина,
    оторванная в область дипломатии, даёт свои колебания на всю дипломатию. А Илья Муромец даёт колебания только на семью на свою. Спичка в библиотеке работает. В
    кинохронику ходят и зажигают в кинохронике большой лист. В библиотеке маленький лист разжигают. Огонь… э-э-э… будет вырабатываться гораздо легче, чем учебник крепкий.
    А крепкий учебник будет весомее, чем гастроном на улице Герцена. А на улице Герцена будет расщеплённый учебник. Тогда учебник будет проходить через улицу Герцена,
    через гастроном номер двадцать два, и замещаться там по формуле экономического единства. Вот в магазине двадцать два она может расщепиться, экономика! На экономистов,
    на диспетчеров, на продавцов, на культуру торговли… Так что, в эту сторону двинется вся экономика. Библиотека двинется в сторону ста двадцати единиц, которые будут…
    э-э-э… предмет укладывать на предмет. Сто двадцать единиц — предмет физика. Электрическая лампочка горит от ста двадцати кирпичей, потому что структура, так сказать,
    похожа у неё на кирпич. Илья Муромец работает на стадионе «Динамо». Илья Муромец работает у себя дома. Вот конкретная дипломатия! Открытая дипломатия — то же самое.
    Ну, берём телевизор, вставляем в Мурманский полуостров, накручиваем там… э-э-э… всё время чёрный хлеб… Так что же, будет Муромец, что ли, вырастать? Илья Муромец,
    что ли, будет вырастать из этого?"""
data = re.sub('\n', ' ', data)

In [3]:
# build dictionaries
idx2sym = dict(enumerate(set(data)))
sym2idx = {v: k for k, v in idx2sym.items()}

In [4]:
class SymbolDataset(torch.utils.data.Dataset):
    def __init__(self, text, window=8):
        self.length = len(text) - window + 1
        self.x = [[sym2idx[s] for s in text[i:i+window]] for i in range(self.length - 1)]
        self.y = [[sym2idx[s] for s in text[i:i+window]] for i in range(1, self.length)]

    def __getitem__(self, index):
        return torch.as_tensor(self.x[index]), torch.as_tensor(self.y[index])
        # return torch.as_tensor(self.x[index]), torch.as_tensor(np.eye(len(sym2idx))[self.y[index]])
    
    def __len__(self):
        return self.length - 1


TEST_FRAC = 0.2
WINDOW = 144
BATCH_SIZE = 128 
# make datasets
bound = math.ceil(len(data) * (1 - TEST_FRAC))

train_dataset = SymbolDataset(data[:bound], window=WINDOW)
valid_dataset = SymbolDataset(data[bound:], window=WINDOW)
# make loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [5]:
class Trainable:
    def fit(self, loader, optim, crit, *, epochs=5, dev='cpu', eval_params=None):
        """
        :param loader - data loader
        :param optim - optimizer
        :param crit - criterion
        :param epochs
        :param device
        :param eval_params - predict() parameters for evaluation
        """
        self.crit = crit
        self.dev = dev
        if not eval_params:
            eval_params = dict()
        self.train()
        
        for ep in range(epochs):
            sum_loss, items = 0.0, 0
            pbar = tqdm(enumerate(loader), total=len(loader), desc=f'Epoch {ep + 1}/{epochs}')
            for i, batch in pbar:
                inputs, labels = batch[0].to(dev), batch[1].to(dev)
                optim.zero_grad()
                outputs = self(inputs)
                # print(outputs.shape, labels.shape)
                # print(outputs)
                # print(labels)
                loss = crit(outputs, labels)
                loss.backward()
                optim.step()

                sum_loss += loss.item()
                items += len(labels)
                pbar.set_postfix({'cumulative loss per item': sum_loss / items})

                # evaluate
                if (i + 1 == len(loader)) and eval_params:
                    self.eval()
                    valid_loader = eval_params.get('valid_loader')
                    pbar.set_postfix_str('calculating final loss...')

                    train_result = self.predict_(loader, **eval_params)
                    report = {'loss': train_result[1], 'metric': 'n/a' if len(train_result) == 2 else train_result[2]}

                    if valid_loader is not None:
                        valid_result = self.predict_(valid_loader, **eval_params)
                        report = {'loss': f'{train_result[1]:03f}/{valid_result[1]:03f}',
                                  'metric': 'n/a' if len(train_result) == 2 else f'{train_result[2]:03f}/{valid_result[2]:03f}'}
                    pbar.set_postfix(report)
                    self.train()
        print('\nDone.')

    def predict_(self, loader, *, metric=None, threshold=0.5, **kwargs):
        if not hasattr(self, 'dev'):
            raise AttributeError('Model is not trained.')
        self.eval()
        loss = 0
        for i, batch in enumerate(loader):
            inputs, labels = batch[0].to(self.dev), batch[1].to(self.dev)
            outputs = self(inputs)
            predicts = torch.cat([predicts, outputs]) if i > 0 else outputs
            true_labels = torch.cat([true_labels, labels]) if i > 0 else labels

            loss += self.crit(outputs, labels).item() / len(loader)
        result = [predicts, loss]
        if metric:
            # pred_labels = (predicts > threshold) * 1            
            result.append(metric(true_labels.cpu(), predicts.detach().cpu()))
        return result

In [6]:
class Net(torch.nn.Module, Trainable):
    def __init__(self, dict_size, embedding_dim=128, drop=0.3, num_classes=1):
        super().__init__()
        self.num_classes = num_classes
        self.embedding = torch.nn.Embedding(dict_size, embedding_dim)
        self.lstm_1 = torch.nn.GRU(embedding_dim, 2 * embedding_dim, num_layers=2, batch_first=True, bidirectional=True)
        # self.lstm_1 = torch.nn.LSTM(embedding_dim, 2 * embedding_dim, num_layers=2, batch_first=True, bidirectional=True)

        self.linear = torch.nn.Linear(2 * embedding_dim, num_classes)
        self.dp = torch.nn.Dropout(drop)
        
    def forward(self, x):
        x = self.embedding(x)
        x, h = self.lstm_1(x)
        x = torch.max_pool1d(x, 2)
        x = self.dp(x)
        x = torch.sigmoid(x)
        x = self.linear(x)
        # x = torch.softmax(x, dim=2)       # по идее же оно тут должно быть - ибо классификация, но с ним обучается ооочень медленно
        return x.permute(0, 2, 1)

In [7]:
DICT_SIZE = len(sym2idx)

torch.manual_seed(11)       # just in case
model = Net(dict_size=DICT_SIZE, embedding_dim=256, drop=0.05, num_classes=DICT_SIZE).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


def metric(true_val, pred_val, mfunc=accuracy_score, **kwargs):
    values = list(map(partial(mfunc, **kwargs), true_val, pred_val.argmax(dim=1)))
    return np.mean(values)

eval_params = {
    'softmax': True,
    'metric': partial(metric, mfunc=f1_score, average='macro'),
    'valid_loader': valid_loader
}
model.fit(train_loader, optimizer, criterion, epochs=5, dev=device, eval_params=eval_params)

Epoch 5/5: 100%|██████████| 12/12 [00:05<00:00,  2.37it/s, loss=0.056578/0.362542, metric=0.982380/0.779703]


Done.


In [10]:
predicts = model.predict_(valid_loader)
detached = predicts[0].detach().cpu()
''.join([idx2sym[i] for i in detached.argmax(dim=1)[:, 0].numpy()])

' потому что структура, так сказать,     похожа у не, на кирпик. Илья Муромец работает на стадионе  финамой. Илья Муромец работает у себя дома. Вот конкретная дипломатий  сткрытая дипломатия — то Ге ссмое.     Ну, бермм телевизор, вставльем в Мурманский полуостров, н'

In [9]:
#